In [1]:
import pandas as pd

# ---------------- Pandas settings --------------- #
# Removes rows and columns truncation of '...'
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

## Analyzing the Saltiness of the Users

In [2]:
## Completely Optional Code below, only try if running Jupyter Lab doesn't find postgres_helper.py
import sys, os
cwd = os.getcwd()
sys.path[0] = cwd[:cwd.rfind('/')]

## Create Connection to PostgreSQL db

In [3]:
from data import postgres_helper as pgh

pg_conn = pgh.create_connection(verbose=True)

Environment file loaded successfully...
Using psycopg2 version: 2.8.4 (dt dec pq3 ext lo64)
Creating Connection to txtqhcho, txtqhcho@salt.db.elephantsql.com...
PostgreSQL database version: ('PostgreSQL 11.5 (Ubuntu 11.5-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0, 64-bit',)


## List all tables in the database

In [4]:
pgh.get_sql_tables(pg_conn)

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,salt,txtqhcho,None,False,False,False,False


## Schema of `salt` table

In [5]:
table_info_query = """SELECT column_name, udt_name, data_type, character_maximum_length
FROM information_schema.columns
WHERE table_name   = 'salt'
;
"""
pgh.select_query(pg_conn, table_info_query)

,column_name,udt_name,data_type,character_maximum_length
0,id,int8,bigint,None
1,by,text,text,None
2,author,text,text,None
3,time,int8,bigint,None
4,time_ts,timestamptz,timestamp with time zone,None
5,text,text,text,None
6,parent,int8,bigint,None
7,salty_score,float8,double precision,None
8,ranking,int8,bigint,None


## List Top 10 rows from the `salt` table

In [6]:
top_10_query = """SELECT * FROM salt LIMIT 10;"""
pgh.select_query(pg_conn, top_10_query)

,id,by,author,time,time_ts,text,parent,salty_score,ranking
0,4895850,Mz,Mz,1355082935,2012-12-09 19:55:35+00:00,"So, basically, you think I have Munchausen the...",4895812,0.3522,670
1,10313701,Mz,Mz,1443725815,2015-10-01 18:56:55+00:00,One way to test your hypothesis is to start re...,10313194,0.1742,670
2,1658291,Mz,Mz,1283476918,2010-09-03 01:21:58+00:00,And then there is always the risk that someone...,1658204,-0.3362,670
3,4911653,Mz,Mz,1355336835,2012-12-12 18:27:15+00:00,Maybe you should acquaint yourself with a book...,4911595,-0.7495,670
4,3996858,Mz,Mz,1337456956,2012-05-19 19:49:16+00:00,"Again, I do not agree. Emotion or ""caring"" is ...",3996819,0.9407,670
5,9894480,Mz,Mz,1437000136,2015-07-15 22:42:16+00:00,Thank you.,9894355,1.0752,670
6,3391384,Mz,Mz,1324838925,2011-12-25 18:48:45+00:00,My mom is a first generation immigrant. I oft...,3391348,0.9549,670
7,9495895,Mz,Mz,1430871869,2015-05-06 00:24:29+00:00,"No, not glib. I understand. This is my version...",9495869,-0.5911,670
8,1584576,Mz,Mz,1281224545,2010-08-07 23:42:25+00:00,"We need a word different than ""homeless"", real...",1584536,0.9633,670
9,4889323,Mz,Mz,1354915543,2012-12-07 21:25:43+00:00,Sorry. There is no intent to be evasive.The th...,4889022,1.0337,670


### Find Top 100 Salty Users by Rank

For `/satly-users` API Endpoint by `GET` Method

In [9]:
pg_conn = pgh.create_connection()
query = """SELECT author, SUM(salty_score) AS total_score, ranking 
FROM salt
GROUP BY author, ranking
ORDER BY total_score ASC
LIMIT 100;"""
pgh.select_query(pg_conn, query)

Environment file loaded successfully...
Using psycopg2 version: 2.8.4 (dt dec pq3 ext lo64)
Creating Connection to txtqhcho, txtqhcho@salt.db.elephantsql.com...
PostgreSQL database version: ('PostgreSQL 11.5 (Ubuntu 11.5-1.pgdg18.04+1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 7.4.0-1ubuntu1~18.04.1) 7.4.0, 64-bit',)


,author,total_score,ranking
0,wnight,-6.3215,1
1,jlgreco,-6.2563,2
2,Karunamon,-5.9359,3
3,Zigurd,-4.4229,4
4,GHFigs,-4.1720,5
5,chrischen,-3.8724,6
6,tsotha,-3.7560,7
7,olefoo,-3.5508,8
8,cpncrunch,-3.5466,9
9,ceejayoz,-3.4976,10


### Top 100 Salty Comments 

`/salty-comments` API Endpoint using `GET` method

In [10]:
pg_conn = pgh.create_connection()
query = """SELECT text, salty_score
FROM salt
ORDER BY salty_score asc
LIMIT 100;"""
pgh.select_query(pg_conn, query)

,text,salty_score
0,So he can't learn from a competitor. Blackberr...,-1.5483
1,"Ugh, that's unbelievably awful.",-1.4794
2,"It's a time delay, meant to discourage furious...",-1.4696
3,No one believes that the US is going to use it...,-1.4234
4,Nonsense. Legals and accounting should definit...,-1.4143
5,>It's just about as bad as plaintext. //Hyperb...,-1.4076
6,You can kill the revolutionary but you can't k...,-1.3970
7,"No, it's because drugs are evil.",-1.3880
8,"I have no idea about the statistics, but in a ...",-1.3801
9,If someone who happens to be Christian carries...,-1.3710


### Get `username` and top 10 salty comments by that user

In [12]:
pg_conn = pgh.create_connection()
query = """SELECT author, text, salty_score
FROM salt
WHERE author = 'wnight'
ORDER BY salty_score asc
LIMIT 10;"""
pgh.select_query(pg_conn, query)

,author,text,salty_score
0,wnight,"It's arbitrary, not legally required, and just...",-1.0804
1,wnight,">> So in other words it's as good as a law, bu...",-1.0748
2,wnight,Why should they be allowed to write onerous co...,-0.8337
3,wnight,"Oh, so you aren't okay with charging a single ...",-0.8138
4,wnight,"Yes, or further, by simply reminding you ""Usin...",-0.7841
5,wnight,It says nothing to people in the know.It takes...,-0.7326
6,wnight,It doesn't matter. You can''t compare some car...,-0.6852
7,wnight,Did you read my post? Even the basic text it s...,-0.5389
8,wnight,And you should do your part first so that you ...,-0.1890
9,wnight,"Yeah, that's what I'd have thought.But I don't...",0.4110
